# Load images with `tf.data`

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/images"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This tutorial provides a simple example of how to load an image dataset using `tf.data`.

The dataset used in this example is distributed as directories of images, with one class of image per directory.

## Setup

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
tf.VERSION

## Download and inspect the dataset

### Retrieve the images

Before you start any training, you'll need a set of images to teach the network about the new classes you want to recognize. We've created an archive of creative-commons licensed flower photos to use initially. 

In [0]:
import pathlib
data_root = tf.keras.utils.get_file('flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', untar=True)
data_root = pathlib.Path(data_root)
print(data_root)

After downloading 218MB, you should now have a copy of the flower photos available:

In [0]:
for item in data_root.iterdir():
  print(item)

In [0]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

len(all_image_paths)

### Inspect the images
Now let's have a quick look at a couple of the images, so we know what we're dealing with:

In [0]:
attributions = (data_root/"LICENSE.txt").read_text().splitlines()[4:]
attributions = [line.split(' CC-BY') for line in attributions]
attributions = dict(attributions)

In [0]:
import IPython.display as display

def caption_image(image_path):
    image_rel = pathlib.Path(image_path).relative_to(data_root)
    return "Image (CC BY 2.0) " + ' - '.join(attributions[str(image_rel)].split(' - ')[:-1])
    

In [0]:
for n in range(3):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(caption_image(image_path))
  print()

### Determine the label for each image

List the available labels:

In [0]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

Assign an index to each label:

In [0]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

Create a list of every file, and its label index

In [0]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

### Load and format the images

TensorFlow includes all the tools you need to load and process images:

In [0]:
img_path = all_image_paths[0]
img_path

here is the raw data:

In [0]:
img_raw = tf.read_file(img_path)
img_raw

Decode it into an image tensor:

In [0]:
img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

Resize it for your model:

In [0]:
img_final = tf.image.resize_images(img_tensor, [192, 192])
img_final = img_final/255.0
print(img_final.shape)
print(img_final.numpy().min())
print(img_final.numpy().max())


Wrap up these up in simple functions for later.

In [0]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize_images(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range

  return image

In [0]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

In [0]:
import matplotlib.pyplot as plt

image_path = all_image_paths[0]
label = all_image_labels[0]

plt.imshow(load_and_preprocess_image(img_path))
plt.grid(False)
plt.xlabel(caption_image(img_path))
plt.title(label_names[label].title())
print()

## Build a `tf.data.Dataset`

### A dataset of images

The easiest way to build a `tf.data.Dataset` is using the `from_tensor_slices` method.

Slicing the array of strings, results in a dataset of strings:

In [0]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)

The `output_shapes` and `output_types` fields describe the content of each item in the dataset. In this case it is a set of scalar binary-strings

In [0]:
print('shape: ', repr(path_ds.output_shapes))
print('type: ', path_ds.output_types)
print()
print(path_ds)

Now create a new dataset that loads and formats images on the fly by mapping `preprocess_image` over the dataset of paths.

In [0]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=8)

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
for n,image in enumerate(image_ds.take(4)):
  plt.subplot(2,2,n+1)
  plt.imshow(image)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.xlabel(caption_image(all_image_paths[n]))

### A dataset of `(image, label)` pairs

Using the same `from_tensor_slices` method we can build a dataset of labels

In [0]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

In [0]:
for label in label_ds.take(10):
  print(label_names[label.numpy()])

Since the datasets are in the same order we can just zip them together to get a dataset of `(image, label)` pairs.

In [0]:
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

The new dataset's `shapes` and `types`, are tuples of shapes and types as well, describing each field:

In [0]:
print('image shape: ', image_label_ds.output_shapes[0])
print('label shape: ', image_label_ds.output_shapes[1])
print('types: ', image_label_ds.output_types)
print()
print(image_label_ds)

### Basic methods for training

To train a model with this dataset you will want the data:

* To be well shuffeled.
* To be batched.
* To repeat forever.
* Batches to available immediately when needed.

These features can be easily added using the `tf.data` api.

In [0]:
BATCH_SIZE = 32

# Setting a shuffle buffer size larger than the dataset ensures that the data is completely shuffled.
ds = image_label_ds.repeat()
ds = ds.shuffle(buffer_size=4000) 
ds = ds.batch(BATCH_SIZE)
# `prefecth` lets the dataset fetches batches, asynchronously while the model is training.
ds = ds.prefetch(1)
ds

### Pipe the dataset to a model

Fetch a copy of MobileNet v2 from `tf.keras.applications`.

This will be used for a simple transfer learning example.

Set the MobileNet weights to be non-trainable:

In [0]:
mobile_net = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable=False

This model expects it's input to be normalized to the `[-1,1]` range:

```
help(keras_applications.mobilenet_v2.preprocess_input)
```

<pre>
...
This function applies the "Inception" preprocessing which converts
the RGB values from [0, 255] to [-1, 1] 
...
</pre>

So before the passing it to the mobilnet model, we need to convert the input from a range of `[0,1]` to `[-1,1]`.

In [0]:
def change_range(image,label):
  image = 2*image-1
  return image, label

keras_ds = ds.map(change_range)

The mobilenet returns a `6x6` spatial grid of features for each image.

Pass it a batch of images to see:

In [0]:
# The dataset may take a few seconds to start, as it fills its shuffle buffer.
image_batch, label_batch = next(iter(keras_ds))

In [0]:
feature_map_batch = mobile_net(image_batch)
print(feature_map_batch.shape)

So build a model wrapped around mobilenet, and use `tf.keras.layers.GlobalAveragePooling2D` to average over those space dimensions, before the output `tf.keras.layers.Dense` layer:

In [0]:
model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(label_names))])

Now it produces outputs of the expected shape:

In [0]:
logit_batch = model(image_batch).numpy()

print("min logit:", logit_batch.min())
print("max logit:", logit_batch.max())
print()

print("Shape:", logit_batch.shape)

Compile the model to describe the training procedure:

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

There are 2 trainable variables: the Dense `weights` and `bias`:

In [0]:
len(model.trainable_variables) 

In [0]:
model.summary()

Train the model.

Normally you would specify the real number of steps per epoch, but for demonstration purposes only run 3 steps.

In [0]:
model.fit(ds, epochs=1, steps_per_epoch=3)  # steps_per_epoch=math.ceil(len(all_image_paths)/BATCH_SIZE)

## Performance

Note: This section just shows a couple of easy tricks that may help performance. For an in depth guide see [Input Pipeline Performance](https://www.tensorflow.org/guide/performance/datasets)

The simple pipeline used above reads each file individually, on each epoch. This is fine for local training on CPU but may not be sufficient for GPU training, and is totally inapprpriate for any sort of distributed training. 

To investigate, first build a simple function to check the performance of our datasets:

In [0]:
import time

def timeit(ds, batches=100):
  overall_start = time.time()
  # Fetch a single batch to prime the pipeline (fill the shuffle buffer),
  # before starting the timer
  it = iter(ds.take(batches+1))
  next(it)

  start = time.time()
  for i,(images,labels) in enumerate(it):
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(batches, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*batches/duration))
  print("Total time: {}s".format(end-overall_start))

The performance of the current dataset is:

In [0]:
ds = image_label_ds.repeat()
ds = ds.shuffle(buffer_size=4000) 
ds = ds.batch(BATCH_SIZE).prefetch(1)
ds

In [0]:
timeit(ds)

### Cache

Use `tf.data.Dataset.cache` to easily take advantage of the performance boost of working with in memory data.

Here the images are cached, after being pre-precessed (decoded and resized):

In [0]:
ds = image_label_ds.cache().repeat()
ds = ds.shuffle(buffer_size=4000) 
ds = ds.batch(BATCH_SIZE).prefetch(1)
ds

In [0]:
timeit(ds)

One disadvantage to using an im memory cache is that the cache must be rebuilt on each run, giving the same startup delay each time the dataset is started:

In [0]:
timeit(ds)

If the data doesn't fit in memory, use a cache file:

In [0]:
ds = image_label_ds.cache(filename='./cache.tf-data')
ds = ds.repeat().shuffle(buffer_size=4000)
ds = ds.batch(BATCH_SIZE).prefetch(1)
ds

In [0]:
timeit(ds)

The cache file also has the advantage that it can be used to quickly restart the dataset without rebuilding the cache. Note how much faster it is the second time:

In [0]:
timeit(ds)

### TFRecord File

TFRecord files are a simple format to store a sequence of binary blobs. In this case the fact that these are read linearly off disk, which may give some performance boost.

It also gives you a portable data file that is apropriate, for example, for streaming over a network.

First, build a `TFRecord` file from the raw image data:

In [0]:
image_ds = tf.data.Dataset.from_tensor_slices(all_image_paths).map(tf.read_file)
tfrec = tf.data.experimental.TFRecordWriter('images.tfrec')
tfrec.write(image_ds)

Next build a dataset that reads from the TFRecord file, and decodes/reformats the images using the `preprocess_image` function we defined earlier.

In [0]:
image_ds = tf.data.TFRecordDataset('images.tfrec').map(preprocess_image, num_parallel_calls=8)

Zip that with the labels dataset we defined earlier, to get the expected `(image,label)` pairs.

In [0]:
ds = tf.data.Dataset.zip((image_ds, label_ds))
ds = ds.repeat().shuffle(buffer_size=4000)
ds=ds.batch(BATCH_SIZE).prefetch(1)
ds

In [0]:
timeit(ds)

This is slower than the `cache` version because we have not cached the preprocessing, which will require more advanced serialization.